In [1]:
########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
np.random.seed(1337)

import pandas as pd
import operator
import sys

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader, CharDataTransformer
from iwillwin.model.sim_zoos import *
import tensorflow as tf
from keras.engine import InputSpec, Layer
from iwillwin.config import dataset_config, model_config
from keras.models import Sequential
import keras.backend as K

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import log_loss

Using TensorFlow backend.
C:\Users\zake7\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\zake7\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
NB_WORDS = 5000
EMBEDDING_DIM = 150
MAX_SEQUENCE_LENGTH = 50
OUT_SIZE = 1

In [3]:
data_transformer = CharDataTransformer(max_num_words=NB_WORDS, max_sequence_length=MAX_SEQUENCE_LENGTH, char_level=False,
                                   normalization=True, features_processed=True)
trains, tests, labels = data_transformer.prepare_data(dual=False)
print("Number of unique words", len(data_transformer.tokenizer.index_docs))
labels = to_categorical(labels)

[DataHelper] Apply normalization on value-type columns


C:\Users\zake7\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Doing preprocessing...
Transforming words to indices...
Shape of data tensor: (320552, 50) (320552, 50)
Shape of label tensor: (320552,)
Preprocessed.
Number of unique words 5196


## Load and prepare the data

In [4]:
print("Embeddings")
print(os.listdir("../data/wordvec"))

Embeddings
['.gitkeep', 'fasttext-50-win3.vec', 'sgns.merge.bigram', 'Tencent_AILab_ChineseEmbedding.txt', 'zh-wordvec-50-cbow-windowsize50.vec', 'zh-wordvec-50-skipgram-windowsize7.vec']


In [5]:
data_loader = DataLoader()
skip_gram_embeddings = data_loader.load_embedding('../data/wordvec/zh-wordvec-50-skipgram-windowsize7.vec')
cbow_embeddings = data_loader.load_embedding('../data/wordvec/zh-wordvec-50-cbow-windowsize50.vec')
fasttext_embeddings = data_loader.load_embedding('../data/wordvec/fasttext-50-win3.vec')

Total 4114 word vectors.
Total 4114 word vectors.
Total 4114 word vectors.


In [6]:
def build_embedding_matrix(embeddings_index, nb_words=NB_WORDS, word_index=data_transformer.tokenizer.word_index):
    #nb_words = min(nb_words, len(embeddings_index))
    #embedding_matrix = np.random.rand(nb_words, 50)
    embedding_matrix = np.zeros((nb_words, 50))
    
    word_index = data_transformer.tokenizer.word_index
    null_words = open('null-word.txt', 'w', encoding='utf-8')

    for word, i in word_index.items():
        if i >= nb_words:
            null_words.write(word + '\n')
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            null_words.write(word + '\n')
    print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
    return embedding_matrix

In [7]:
cbow_matrix = build_embedding_matrix(cbow_embeddings)
skipgram_matrix = build_embedding_matrix(skip_gram_embeddings)
fasttext_matrix = build_embedding_matrix(fasttext_embeddings)
meta_embeddings = np.concatenate((cbow_matrix, skipgram_matrix, fasttext_matrix), axis=1)
meta_embeddings[0] = np.array([0] * 150) # zero padding

Null word embeddings: 948
Null word embeddings: 948
Null word embeddings: 948


# Add tricky features

## Rumor words

In [8]:
train_df = pd.read_csv('../data/dataset/train.csv')
test_df = pd.read_csv('../data/dataset/test.csv')

In [9]:
rumor_words = ['辟谣', '谣言', '勿传', '假的']

def is_rumor(text):
    if type(text) != str:
        print(text, type(text))
        return 0
    for rumor_word in rumor_words:
        if rumor_word in text:
            return 1
    return 0

def has_split_symbol(text):
    if type(text) != str:
        return 0
    if '|' in text:
        return 1
    return 0

for df in [train_df, test_df]:
    df['has_|'] = df['title2_zh'].apply(has_split_symbol)
    df['has_rumor_words'] = df['title2_zh'].apply(is_rumor)

nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>


In [10]:
train_has_rumor = train_df.has_rumor_words.values
test_has_rumor = test_df.has_rumor_words.values

trick_trains_features = np.concatenate((trains[2], train_has_rumor.reshape((-1, 1))), axis=1)
trick_tests_features = np.concatenate((tests[2], test_has_rumor.reshape((-1, 1))), axis=1)

## Exact Match

In [11]:
def _build_exact_match_sequences(sent_1, sent_2):
    sent_1_char_set = set(sent_1)
    sent_2_char_set = set(sent_2)
    intersection = sent_1_char_set & sent_2_char_set
    
    sent_1_em = np.zeros_like(sent_1)
    sent_2_em = np.zeros_like(sent_2)

    for i in range(len(sent_1)):
        if sent_1[i] == 0:
            continue
        if sent_1[i] in intersection:
            sent_1_em[i] = 1
    
    for i in range(len(sent_2)):
        if sent_2[i] == 0:
            continue        
        if sent_2[i] in intersection:
            sent_2_em[i] = 1
    
    return sent_1_em, sent_2_em

def build_exact_match_sequences(sents_1, sents_2):
    sents_1_em, sents_2_em = [], []
    for sent_1, sent_2 in zip(sents_1, sents_2):
        sent_1_em, sent_2_em = _build_exact_match_sequences(sent_1, sent_2)
        sents_1_em.append(sent_1_em)
        sents_2_em.append(sent_2_em)
    return np.array(sents_1_em), np.array(sents_2_em)

In [12]:
%%time
trains_1_ems, trains_2_ems = build_exact_match_sequences(trains[0], trains[1])
tests_1_ems, tests_2_ems = build_exact_match_sequences(tests[0], tests[1])

Wall time: 17.6 s


In [13]:
print("Shape of train em", trains_1_ems.shape, trains_2_ems.shape)
print("Shape of test em", tests_1_ems.shape, tests_2_ems.shape)

Shape of train em (320552, 50) (320552, 50)
Shape of test em (80126, 50) (80126, 50)


In [14]:
em_train_features = (trains_1_ems, trains_2_ems)
em_test_features = (tests_1_ems, tests_2_ems)

# Trick or Treat!!!

In [15]:
use_tricky = True

if use_tricky:
    trains = (trains[0], trains[1], trick_trains_features)
    tests = (tests[0], tests[1], trick_tests_features)
else:
    trains = (trains[0], trains[1], trains[2])
    tests = (tests[0], tests[1], tests[2])

In [16]:
import numpy as np
import pandas as pd
import importlib

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint

from iwillwin.config import model_config

class ModelTrainer(object):

    def __init__(self, model_stamp, epoch_num, learning_rate=1e-3,
                 shuffle_inputs=False, verbose_round=40, early_stopping_round=8):
        self.models = []
        self.model_stamp = model_stamp
        self.val_loss = -1
        self.auc = -1
        self.epoch_num = epoch_num
        self.learning_rate = learning_rate
        self.eps = 1e-10
        self.verbose_round = verbose_round
        self.early_stopping_round = early_stopping_round
        self.shuffle_inputs = shuffle_inputs
        self.class_weight = [0.93, 1.21]

    def train_folds(self, X, y, fold_count, em_train_features, batch_size, get_model_func, augments=None, skip_fold=0, patience=10, scale_sample_weight=False,
                    class_weight=None, self_aware=False, swap_input=False):
        X1, X2, features, = X
        em1, em2 = em_train_features
        weight_val=scale_sample_weight

        fold_size = len(X1) // fold_count
        models = []
        fold_predictions = []
        score = 0

        for fold_id in range(0, fold_count):
            fold_start = fold_size * fold_id
            fold_end = fold_start + fold_size

            if fold_id == fold_count - 1:
                fold_end = len(X1)

            train_x1 = np.concatenate([X1[:fold_start], X1[fold_end:]])
            train_x2 = np.concatenate([X2[:fold_start], X2[fold_end:]])
            train_features = np.concatenate([features[:fold_start], features[fold_end:]])
            
            train_em_1 = np.concatenate([em1[:fold_start], em1[fold_end:]])
            train_em_2 = np.concatenate([em2[:fold_start], em2[fold_end:]])
            
            train_y = np.concatenate([y[:fold_start], y[fold_end:]])

            val_x1 = X1[fold_start:fold_end]
            val_x2 = X2[fold_start:fold_end]
            val_features = features[fold_start:fold_end]
            val_em1 = em1[fold_start:fold_end]
            val_em2 = em2[fold_start:fold_end]
            val_y = y[fold_start:fold_end]

            fold_pos = (np.sum(train_y) / len(train_x1))

            train_data = {
                "first_sentences": train_x1,
                "second_sentences": train_x2,
                "mata-features": train_features,
                "first_exact_match": train_em_1,
                "second_exact_match": train_em_2,
            }

            val_data = {
                "first_sentences": val_x1,
                "second_sentences": val_x2,
                "mata-features": val_features,
                "first_exact_match": val_em1,
                "second_exact_match": val_em2,
            }

            model, bst_val_score, fold_prediction = self._train_model_by_logloss(
                get_model_func(), batch_size, train_data, train_y, val_data, val_y, fold_id, patience, class_weight, weight_val=None)
    
            score += bst_val_score
            models.append(model)
            fold_predictions.append(fold_prediction)

        self.models = models
        self.val_loss = score / fold_count
        return models, self.val_loss, fold_predictions

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience):
        # return a list which holds [models, val_loss, auc, prediction]
        raise NotImplementedError

class KerasModelTrainer(ModelTrainer):

    def __init__(self, *args, **kwargs):
        super(KerasModelTrainer, self).__init__(*args, **kwargs)
        pass

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience, class_weight, weight_val):
        early_stopping = EarlyStopping(monitor='val_loss', patience=patience)
        bst_model_path = self.model_stamp + str(fold_id) + '.h5'
        val_data = (val_x, val_y, weight_val) if weight_val is not None else (val_x, val_y)
        model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
        hist = model.fit(train_x, train_y,
                         validation_data=val_data,
                         epochs=self.epoch_num, batch_size=batch_size, shuffle=True,
                         callbacks=[early_stopping, model_checkpoint],
                         class_weight=class_weight)
        bst_val_score = max(hist.history['val_weighted_accuracy']) # note this is the hard version
        model.load_weights(bst_model_path)
        predictions = model.predict(val_x)

        return model, bst_val_score, predictions

# Path setting

In [18]:
oofs_dir = "../data/oofs/"
output_dir = "../data/output/"
onehot_pred_dir = "../data/one_hot_pred/"

In [24]:
def make_predictions():
    # REFACTOR: remove this lazy solution
    oofs_path = oofs_dir + model_submit_prefix
    output_path = output_dir + model_submit_prefix
    one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

    print("Predicting training results...")
    train_predicts = np.concatenate(folds_preds, axis=0)
    oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
    submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    oofs.to_csv(submit_path, index=False)

    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict({"first_sentences":tests[0],
                                       "second_sentences":tests[1],
                                       "mata-features":tests[2],
                                       "first_exact_match": em_test_features[0],
                                       "second_exact_match": em_test_features[1],
                                      }, batch_size=128, verbose=1)
        test_predicts_list.append(test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    for fold_predict in test_predicts_list:
        test_predicts += fold_predict
    test_predicts /= len(test_predicts_list)

    test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
    submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    test_predicts.to_csv(submit_path, index=False)

    print("Predicting labeled testing results...")
    ids = pd.read_csv("../data/dataset/test.csv")
    pred_labels = test_predicts.idxmax(axis=1)
    sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
    submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    sub = sub.to_csv(submit_path, index=False)

# DenseRNN

In [ ]:
fold_count = 8
embedding_matrix=meta_embeddings
models_checkpoints_path = "3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM"
model_submit_prefix = "3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM"

def _agent_get_model():
    return get_char_darnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)

test_predicts_list = []
oofs_predictions = []
pre_trained_models = []

trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
models, score, folds_preds = trainer.train_folds(X=trains, y=labels, augments=None, fold_count=fold_count, batch_size=128,
    em_train_features=em_train_features, get_model_func=_agent_get_model, patience=25)

print("score", score)
make_predictions()

# DenseCNN

In [ ]:
fold_count = 8
models_checkpoints_path = "3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM"
model_submit_prefix = "3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM"

def _agent_get_model():
    return get_char_dense_cnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)

test_predicts_list = []
oofs_predictions = []
pre_trained_models = []

trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
models, score, folds_preds = trainer.train_folds(X=trains, y=labels, augments=None, fold_count=fold_count, batch_size=1024,
    em_train_features=em_train_features, get_model_func=_agent_get_model, patience=25)

print("score", score)
make_predictions()

# ESIM

In [32]:
fold_count = 8
models_checkpoints_path = "3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM"
model_submit_prefix = "3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM"

def _agent_get_model():
    return get_char_ESIM(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)

test_predicts_list = []
oofs_predictions = []
pre_trained_models = []

trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
models, score, folds_preds = trainer.train_folds(X=trains, y=labels, augments=None, fold_count=fold_count, batch_size=128,
    em_train_features=em_train_features, get_model_func=_agent_get_model,  patience=10)

print("score", score)
make_predictions()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_sentences (InputLayer)    (None, 50)           0                                            
__________________________________________________________________________________________________
second_sentences (InputLayer)   (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 50, 150)      750000      first_sentences[0][0]            
                                                                 second_sentences[0][0]           
__________________________________________________________________________________________________
spatial_dropout1d_5 (SpatialDro (None, 50, 150)      0           embedding_5[0][0]                
__________

Train on 280483 samples, validate on 40069 samples
Epoch 1/500


KeyboardInterrupt: 